In [ ]:
-- Primeira Query - Diferença entre crimes diários e noturnos
DROP VIEW IF EXISTS Numero_crimes_diarios
DROP VIEW IF EXISTS Numero_crimes_noturnos

CREATE VIEW Numero_crimes_diarios AS
    SELECT Q.ID, COUNT(*) N_CRIMES_DIARIOS
    FROM Crimes C, Quads Q
    WHERE C.QUAD = Q.ID AND (C.PERIODO_OCORRENCIA = 'PELA MANHÃƒ' OR C.PERIODO_OCORRENCIA = 'A TARDE')
    GROUP BY Q.ID;

CREATE VIEW Numero_crimes_noturnos AS
    SELECT Q.ID, COUNT(*) N_CRIMES_NOTURNOS
    FROM Crimes C, Quads Q
    WHERE C.QUAD = Q.ID AND (C.PERIODO_OCORRENCIA = 'A NOITE' OR C.PERIODO_OCORRENCIA = 'DE MADRUGADA')
    GROUP BY Q.ID;


DROP VIEW IF EXISTS Numero_crimes

CREATE VIEW Numero_crimes AS
    SELECT Q.ID, COUNT(*) N_CRIMES
    FROM Crimes C, Quads Q
    WHERE Q.ID = C.QUAD
    GROUP BY Q.ID;


SELECT NCN.ID, NCD.N_CRIMES_DIARIOS, NCN.N_CRIMES_NOTURNOS
    FROM Numero_crimes_noturnos NCN, Numero_crimes_diarios NCD
    WHERE NCN.ID = NCD.ID
    ORDER BY NCD.N_CRIMES_DIARIOS, NCN.N_CRIMES_NOTURNOS;


SELECT NCN.ID, CAST(NCD.N_CRIMES_DIARIOS AS FLOAT)/NC.N_CRIMES*100 Per_diario, CAST(NCN.N_CRIMES_NOTURNOS AS FLOAT)/NC.N_CRIMES*100 Per_noturno
    FROM Numero_crimes_noturnos NCN, Numero_crimes_diarios NCD, Numero_crimes NC
    WHERE NCN.ID = NCD.ID AND NCN.ID = NC.ID;

In [ ]:
-- Segunda Query - Efeito dos postes em crimes noturnos
DROP VIEW IF EXISTS Numero_postes

CREATE VIEW Numero_postes AS
    SELECT Q.ID, COUNT(*) N_POSTES
    FROM postes_ilum P, Quads Q
    WHERE P.QUAD = Q.ID
    GROUP BY Q.ID;

SELECT DISTINCT NCN.ID, NCN.N_CRIMES_NOTURNOS, NP.N_POSTES
    FROM Numero_crimes_noturnos NCN, Numero_postes NP
    WHERE NCN.ID = NP.ID
    ORDER BY NCN.N_CRIMES_NOTURNOS;

In [ ]:
-- Terceira Query - Efeito dos postos policiais em crimes

DROP VIEW IF EXISTS Numero_postos

CREATE VIEW Numero_postos AS
    SELECT Q.ID, COUNT(*) N_POSTOS
    FROM Postos P, Quads Q
    WHERE P.QUAD= Q.ID
    GROUP BY Q.ID;

SELECT NC.ID, NC.N_CRIMES, NP.N_POSTOS
    FROM Numero_crimes NC, Numero_postos NP
    WHERE NC.ID = NP.ID
    ORDER BY NC.N_CRIMES;

In [ ]:
-- Quarta Query - Tipos de crimes mais comum de manhã

CREATE VIEW Crimes_manhã AS
SELECT C.TIPO_CRIME, COUNT(*) Contagem
    FROM Crimes C
    WHERE C.PERIODO_OCORRENCIA = 'PELA MANHÃƒ'
    GROUP BY C.TIPO_CRIME;

SELECT TIPO_CRIME
    FROM Crimes_manhã
    WHERE Contagem = (SELECT MAX(Contagem) FROM Crimes_manhã);

In [ ]:
-- Quinta Query - Tipo de crime que mais acontece na região com mais postos policiais

SELECT C.TIPO_CRIME, COUNT(*) Contagem
    FROM Crimes C
    WHERE C.QUAD IN (SELECT NP.ID
            FROM Numero_postos NP
            WHERE N_POSTOS = (SELECT MAX(N_POSTOS) FROM Numero_postos))
    GROUP BY C.TIPO_CRIME
    ORDER BY Contagem DESC

In [ ]:
ALTER TABLE quads
ADD ZONA VARCHAR(8) NOT NULL DEFAULT 'FORA';

UPDATE quads
SET ZONA = 'SUL' WHERE (LAT_MIN >= -23.9480184 AND LAT_MIN < -23.577875 AND LON_MIN >= -46.822750808 AND LON_MIN <= -46.25966774);

UPDATE quads
SET ZONA = 'NORTE' WHERE (LAT_MIN > -23.497775 AND LAT_MIN <= -23.326397399999998 AND LON_MIN >= -46.67303056 AND LON_MIN <= -46.255183808000005);

UPDATE quads
SET ZONA = 'CENTRAL' WHERE (LAT_MIN >= -23.577875 AND LAT_MIN <= -23.497775 AND LON_MIN >= -46.67303056 AND LON_MIN <= -46.557680);

UPDATE quads
SET ZONA = 'LESTE' WHERE (LAT_MIN >= -23.9480184 AND LAT_MIN < -23.47150556 AND LON_MIN > -46.557680 AND LON_MIN <= -46.255183808000005);

UPDATE quads
SET ZONA = 'OESTE' WHERE (LAT_MIN >= -23.577875 AND LAT_MIN <= -23.3263974 AND LON_MIN >= -46.822750808 AND LON_MIN < -46.67303056);

In [ ]:
-- Equipamentos de Segurança por Região

CREATE VIEW POSTOS_REG AS
SELECT Q.Zona, SUM(NP.N_POSTOS) N_POSTOS
    FROM QUADS Q, Numero_postos NP
    WHERE Q.ID = NP.ID
    GROUP BY Q.ZONA

CREATE VIEW POSTES_REG AS
SELECT Q.Zona, SUM(NP.N_POSTES) N_POSTES
    FROM QUADS Q, Numero_postes NP
    WHERE Q.ID = NP.ID
    GROUP BY Q.ZONA

SELECT POR.ZONA, POR.N_POSTOS, PER.N_POSTES
    FROM POSTOS_REG POR, POSTES_REG PER
    WHERE POR.ZONA = PER.ZONA

In [ ]:
DROP VIEW IF EXISTS Tipo_Regiao;
DROP VIEW IF EXISTS Crimes_Regiao;

CREATE VIEW Tipo_Regiao AS
SELECT Q.ZONA, TC.NOME, COUNT(*) Contagem
    FROM QUADS Q, CRIMES C, TIPOS_CRIMES TC
    WHERE Q.ID = C.QUAD AND TC.NOME = C.TIPO_CRIME
    GROUP BY Q.ZONA, TC.NOME;
    
SELECT ZONA, NOME, Contagem
    FROM Tipo_Regiao
    WHERE Contagem = (SELECT MAX(CONTAGEM) FROM Tipo_Regiao WHERE ZONA = 'NORTE')
    OR  Contagem = (SELECT MAX(CONTAGEM) FROM Tipo_Regiao WHERE ZONA = 'SUL')
    OR  Contagem = (SELECT MAX(CONTAGEM) FROM Tipo_Regiao WHERE ZONA = 'LESTE')
    OR  Contagem = (SELECT MAX(CONTAGEM) FROM Tipo_Regiao WHERE ZONA = 'OESTE')
    OR  Contagem = (SELECT MAX(CONTAGEM) FROM Tipo_Regiao WHERE ZONA = 'CENTRAL');

In [ ]:
-- Região com mais Homicídio

SELECT TR.ZONA, TR.NOME, TR.CONTAGEM
    FROM Tipo_Regiao TR
    WHERE TR.CONTAGEM = (SELECT MAX(CONTAGEM) FROM Tipo_Regiao WHERE NOME = 'HOMICÃDIO') AND TR.NOME = 'HOMICÃDIO'

In [ ]:
-- CRIMES E EQUIPAMENTOS DE SEGURANÇA POR REGIÃO
CREATE VIEW CRIMES_REG AS
SELECT Q.ZONA, COUNT(*) Contagem
    FROM QUADS Q, CRIMES C
    WHERE Q.ID = C.QUAD
    GROUP BY Q.ZONA;

-- Contribuição de cada região para números de crimes
SELECT Zona, (CAST(Contagem AS FLOAT)/(SELECT CAST(COUNT(*) AS FLOAT) FROM CRIMES))*100 Porcentagem
    FROM CRIMES_REG

-- Comparação entre distribuição de crimes e postos
SELECT PR.Zona, (CAST(PR.N_POSTOS AS FLOAT)/(SELECT CAST(COUNT(*) AS FLOAT) FROM POSTOS))*100 Postos, 
	(CAST(CR.Contagem AS FLOAT)/(SELECT CAST(COUNT(*) AS FLOAT) FROM CRIMES))*100 Crimes
	FROM POSTOS_REG PR, CRIMES_REG CR
	WHERE PR.ZONA = CR.ZONA

-- Comparação entre distribuição de crimes e postes
SELECT PR.Zona, (CAST(PR.N_POSTES AS FLOAT)/(SELECT CAST(COUNT(*) AS FLOAT) FROM POSTES_ILUM))*100 Postes, 
	(CAST(CR.Contagem AS FLOAT)/(SELECT CAST(COUNT(*) AS FLOAT) FROM CRIMES))*100 Crimes
	FROM POSTES_REG PR, CRIMES_REG CR
	WHERE PR.ZONA = CR.ZONA

-- Comparação entre distribuição de crimes, postos e postes
SELECT POR.Zona, (CAST(POR.N_POSTOS AS FLOAT)/(SELECT CAST(COUNT(*) AS FLOAT) FROM POSTOS))*100 Postos, 
	(CAST(PER.N_POSTES AS FLOAT)/(SELECT CAST(COUNT(*) AS FLOAT) FROM postes_ilum))*100 Postes, 
	(CAST(CR.Contagem AS FLOAT)/(SELECT CAST(COUNT(*) AS FLOAT) FROM CRIMES))*100 Crimes
	FROM POSTOS_REG POR, CRIMES_REG CR, POSTES_REG PER
	WHERE POR.ZONA = CR.ZONA AND POR.ZONA = PER.ZONA